# Perform univariate and multivariate anomaly detection on the `Sales` and `Profit` columns of the Superstore dataset.

* Use the `Local Outlier Factor` algorithm.
* Write in detail how this algorithm helps us in detecting anomalies.

[Resource](https://scikit-learn.org/stable/modules/outlier_detection.html)




In [1]:
# !pip install --upgrade --force-reinstall xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
  Attempting uninstall: xlrd
    Found existing installation: xlrd 2.0.1
    Uninstalling xlrd-2.0.1:
      Successfully uninstalled xlrd-2.0.1


In [2]:
# Importing libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# pip install --upgrade xlrd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Reading data
# path = 'E:/Data Science/amlabetter/Module 4/Week 5/Day 1/Superstore.xls'
path = '/content/drive/MyDrive/Almabetter/Cohort Nilgiri/Module 4/Week 5/Day 1/Superstore.xls'
super_store_df = pd.read_excel(path)
super_store_df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [5]:
# df for Isolation Forest
df = super_store_df[['Sales','Profit']]
# df for Local outlier factor
lof_df = df.copy()
del super_store_df
df.head()

,Sales,Profit
0,261.9600,41.9136
1,731.9400,219.5820
2,14.6200,6.8714
3,957.5775,-383.0310
4,22.3680,2.5164


###  <h1> Univariate anamoly detection

###  Isolation Forest

In [6]:
# Isolation forest detecting Sales column
from sklearn.ensemble import IsolationForest
isf_model = IsolationForest(n_estimators=200, random_state=0, verbose=2, contamination=.1)
df['sales_pred'] = isf_model.fit_predict(df['Sales'].values.reshape(-1,1))
df['sales_score'] = isf_model.decision_function(df['Sales'].values.reshape(-1,1))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Building estimator 1 of 200 for this parallel run (total 200)...
Building estimator 2 of 200 for this parallel run (total 200)...
Building estimator 3 of 200 for this parallel run (total 200)...
Building estimator 4 of 200 for this parallel run (total 200)...
Building estimator 5 of 200 for this parallel run (total 200)...
Building estimator 6 of 200 for this parallel run (total 200)...
Building estimator 7 of 200 for this parallel run (total 200)...
Building estimator 8 of 200 for this parallel run (total 200)...
Building estimator 9 of 200 for this parallel run (total 200)...
Building estimator 10 of 200 for this parallel run (total 200)...
Building estimator 11 of 200 for this parallel run (total 200)...
Building estimator 12 of 200 for this parallel run (total 200)...
Building estimator 13 of 200 for this parallel run (total 200)...
Building estimator 14 of 200 for this parallel run (total 200)...
Building estimator 15 of 200 for this parallel run (total 200)...
Building estimator 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished


In [7]:
# percentage of detected anamoly according to sales column
round((len(df[ (df['sales_pred'] == -1)])/df.shape[0])*100,2)

10.01

In [8]:
# Isolation forest detecting Profit column
df['profit_pred'] = isf_model.predict(df['Profit'].values.reshape(-1,1))
df['profit_score'] = isf_model.decision_function(df['Profit'].values.reshape(-1,1))

In [9]:
# percentage of detected anamoly according to sales column
round((len(df[ (df['profit_pred'] == -1)])/df.shape[0])*100,2)

1.02

In [10]:
df.head()

,Sales,Profit,sales_pred,sales_score,profit_pred,profit_score
0,261.9600,41.9136,1,0.055517,1,0.120372
1,731.9400,219.5820,-1,-0.033255,1,0.057265
2,14.6200,6.8714,1,0.139866,1,0.125020
3,957.5775,-383.0310,-1,-0.064311,1,0.052567
4,22.3680,2.5164,1,0.130186,1,0.097473


### Local Outlier Factor

In [11]:
# Sales column
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor(n_neighbors=20,contamination=.1, novelty=True)
clf.fit(df['Sales'].values.reshape(-1,1))
lof_df['sales_pred'] = clf.predict(df['Sales'].values.reshape(-1,1))
lof_df['sales_score'] = clf.decision_function(df['Sales'].values.reshape(-1,1))

In [12]:
# Profit column
lof_df['profit_pred'] = clf.predict(df['Profit'].values.reshape(-1,1))
lof_df['profit_score'] = clf.decision_function(df['Profit'].values.reshape(-1,1))

In [13]:
lof_df.head()

,Sales,Profit,sales_pred,sales_score,profit_pred,profit_score
0,261.9600,41.9136,1,0.283542,1,0.396555
1,731.9400,219.5820,-1,-0.005679,1,0.311772
2,14.6200,6.8714,1,0.335340,1,0.342345
3,957.5775,-383.0310,1,0.353933,-1,-1564.284490
4,22.3680,2.5164,1,0.346261,1,0.232682


### <h1> Multivariate anamoly detection

### Isolation Forest

In [14]:
# Isolation forest
from sklearn.ensemble import IsolationForest
isf_model = IsolationForest(n_estimators=20, contamination=.1, random_state=2, verbose=2)
df['multi_var_pred'] = isf_model.fit_predict(df[['Sales','Profit']])
df['multi_var_score'] = isf_model.decision_function(df[['Sales','Profit']])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


Building estimator 1 of 20 for this parallel run (total 20)...
Building estimator 2 of 20 for this parallel run (total 20)...
Building estimator 3 of 20 for this parallel run (total 20)...
Building estimator 4 of 20 for this parallel run (total 20)...
Building estimator 5 of 20 for this parallel run (total 20)...
Building estimator 6 of 20 for this parallel run (total 20)...
Building estimator 7 of 20 for this parallel run (total 20)...
Building estimator 8 of 20 for this parallel run (total 20)...
Building estimator 9 of 20 for this parallel run (total 20)...
Building estimator 10 of 20 for this parallel run (total 20)...
Building estimator 11 of 20 for this parallel run (total 20)...
Building estimator 12 of 20 for this parallel run (total 20)...
Building estimator 13 of 20 for this parallel run (total 20)...
Building estimator 14 of 20 for this parallel run (total 20)...
Building estimator 15 of 20 for this parallel run (total 20)...
Building estimator 16 of 20 for this parallel run

In [15]:
# Percentage of multivariate anamoly
round((len(df[df.multi_var_pred == -1])/df.shape[0])*100,2)

10.0

In [16]:
df.head()

,Sales,Profit,sales_pred,sales_score,profit_pred,profit_score,multi_var_pred,multi_var_score
0,261.9600,41.9136,1,0.055517,1,0.120372,1,0.095579
1,731.9400,219.5820,-1,-0.033255,1,0.057265,-1,-0.044596
2,14.6200,6.8714,1,0.139866,1,0.125020,1,0.190917
3,957.5775,-383.0310,-1,-0.064311,1,0.052567,-1,-0.149442
4,22.3680,2.5164,1,0.130186,1,0.097473,1,0.185347


In [17]:
df[ (df['sales_pred']== -1) & (df['profit_pred'] == -1) & (df['multi_var_pred'] == -1)].head(10)

,Sales,Profit,sales_pred,sales_score,profit_pred,profit_score,multi_var_pred,multi_var_score
149,1951.840,585.5520,-1,-0.179392,-1,-0.000053,-1,-0.215870
244,2001.860,580.5394,-1,-0.186425,-1,-0.001386,-1,-0.218361
251,3347.370,636.0003,-1,-0.258328,-1,-0.009839,-1,-0.283449
318,3991.980,1995.9900,-1,-0.274302,-1,-0.185451,-1,-0.328089
353,4355.168,1415.4296,-1,-0.283929,-1,-0.116540,-1,-0.328089
392,1799.970,701.9883,-1,-0.166156,-1,-0.024673,-1,-0.233484
509,6354.950,3177.4750,-1,-0.305232,-1,-0.254584,-1,-0.328089
515,2999.950,1379.9770,-1,-0.248207,-1,-0.112367,-1,-0.319534
814,1399.930,601.9699,-1,-0.115623,-1,-0.003750,-1,-0.205991
910,1928.780,829.3754,-1,-0.177704,-1,-0.045584,-1,-0.243737


### Local Outlier Factor

In [18]:
# Sales column
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor(n_neighbors=20,contamination=.1, novelty=True)
clf.fit(lof_df[['Sales', 'Profit']])
lof_df['multi_var_pred'] = clf.predict(lof_df[['Sales','Profit']])
lof_df['multi_var_score'] = clf.decision_function(lof_df[['Sales','Profit']])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  "X does not have valid feature names, but"


In [19]:
lof_df.head()

,Sales,Profit,sales_pred,sales_score,profit_pred,profit_score,multi_var_pred,multi_var_score
0,261.9600,41.9136,1,0.283542,1,0.396555,1,0.141830
1,731.9400,219.5820,-1,-0.005679,1,0.311772,1,0.074858
2,14.6200,6.8714,1,0.335340,1,0.342345,1,0.214249
3,957.5775,-383.0310,1,0.353933,-1,-1564.284490,-1,-0.027050
4,22.3680,2.5164,1,0.346261,1,0.232682,1,0.201005


In [20]:
lof_df[(lof_df['sales_pred']==-1) & (lof_df['profit_pred']==-1) & (lof_df['multi_var_pred']==-1) ].head(10)

,Sales,Profit,sales_pred,sales_score,profit_pred,profit_score,multi_var_pred,multi_var_score
165,8159.952,-1359.9920,-1,-1.309234e+00,-1,-5.547012e+03,-1,-1.314465
211,20.800,6.5000,-1,-1.733861e+00,-1,-1.999982e+08,-1,-0.098940
375,479.040,-29.9400,-1,-1.980651e+01,-1,-1.248562e+02,-1,-0.048880
494,189.882,-94.9410,-1,-7.134132e-01,-1,-3.898425e+02,-1,-0.079776
683,7999.980,-3839.9904,-1,-1.263363e+00,-1,-1.565710e+04,-1,-2.134882
869,1141.470,-760.9800,-1,-1.240015e-01,-1,-3.105050e+03,-1,-0.234146
919,12.992,-0.8120,-1,-1.561500e+08,-1,-6.111553e+00,-1,-0.352498
1680,42.368,8.4736,-1,-1.946992e-01,-1,-2.728555e-01,-1,-0.207337
2090,26.550,13.0095,-1,-1.278493e-01,-1,-1.180625e+08,-1,-0.200616
2371,60.288,-27.1296,-1,-1.151251e+00,-1,-1.133992e+02,-1,-0.234752
